# Data Anaysis Project part1:

I am using Vancouver rental ads on craigslist to analyze rental market. Later, I will create the class to be able to analyze dynamically. Also, I will start collecting the data and save them to the CSV, so I can analyze the trend of the rental market as well. 

I will follow what I am learning from Google Data Analytics course to complete this project.

* Ask
* Prepare
* Process
* Analyze
* Share
* Act



## Overview of the project:

Analyze housing rental market in Fraser Valley to find out what is hot to real estate investors in this area. Iam going to create the code to analyze dynamically so the investors can apply in their area for their own use.  


## ASK 
    
    Find out what is 'HOT' in the market for investor, so i can find a property for them. Find out supply and demand of the rental housing market and calculate appropreate purchase price. 

#### What are the questions to ask?
Supply side:
* How much is the market rental price?
* What type of the rental is most popular and least popular?
* How many ads per day? Number of supply 
* Is rental price rising or decreasing over time?
* What is the correlation between housing price and rental price?
* From those questions, what is my prediction of the rental market? 

Demand side:
* What kind of propety does people look for rent?
* What is the price range they are looking for?
* What are the purposes of renting. (Who is looking for rent)





## Prepare  

#### Using Craigslist add to collect market data: (Supply side)
Scrape craigslist rental housing pages and store in csv file. 

<a href='https://github.com/rileypredum/East-Bay-Housing-Web-Scrape/blob/master/EB_Apt_Prices_Final.ipynb'>Code: I will use this code and modify for my situation.  </a> <br>
<a href='https://towardsdatascience.com/web-scraping-craigslist-a-complete-tutorial-c41cea4f4981'>Explaning</a>

## Process

I noticed that the all the values are string, so I will convert to correct data type as I collect the data. 

In [200]:
# Import libralis
from requests import get
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np

from time import sleep
import re
from random import randint
from warnings import warn
from time import time
from IPython.display import clear_output

In [2]:
def get_url(city_name, num_post):
    return f'https://{city_name.lower()}.craigslist.org/search/apa?s={num_post}'

In [8]:
# Chilliwack rental housing

response = get(get_url(city_name = 'Abbotsford', num_post = 120))


soup = BeautifulSoup(response.text, 'html.parser')

posts = soup.find_all('li', class_='result-row')
print(type(posts)) # to double check that I got resultSet
print(len(posts)) # to double check I got 120 elements

<class 'bs4.element.ResultSet'>
120


In [221]:
%%time
# Find the total number of posts to find the limit of the pagination
city = 'Vancouver'
response = get(get_url(city_name = city, num_post=0))
soup = BeautifulSoup(response.text, 'html.parser')
posts = soup.find_all('li', class_='result-row')
results_num = soup.find('div', class_='search-legend')
results_total = int(results_num.find('span', class_='totalcount').text)
print(f"Total number of search result: {results_total}")

# each page has 119 posts so each new page is defined as follows; s=120, s=240, .....
pages = np.arange(0, results_total+1, 120)

iterations = 0

post_date = []
post_cities = []
num_bedroom = []
sqfts = []
prices = []
post_titles = []
post_links = []

for page in pages:
    
    # get request
    response = get(get_url(city_name=city, num_post=page))
    
    sleep(1)
    
    # throw warning for status code that are not 200
    if response.status_code != 200:
        warn(f'Request: {requests}; Status code: {response.status_code}')
        
    soup = BeautifulSoup(response.text, 'html.parser')
    
    posts = soup.find_all('li', class_='result-row')
    
    
    # Extract data 
    for post in posts:
        
        if post.find('span', class_ = 'result-hood') is not None:
            
            # Posting date
            post_datetime = post.find('time', class_='result-date')['datetime']
            post_date.append(post_datetime)
            
            # Neighbourhoods
            post_city = post.find('span', class_='result-hood').text.strip('( )')
            post_cities.append(post_city)
            
            # title text
            post_title = post.find('a', class_='result-title hdrlnk')
            post_title_text = post_title.text
            post_titles.append(post_title_text)
            
            # Post price in integer
            price = int(post.find('span', class_='result-price').text.split('$')[1].replace(',', ''))
            prices.append(price)
            
            # Post link
            post_link = post_title['href']
            post_links.append(post_link)
            
            if post.find('span', class_ = 'housing') is not None:
                
                #if the first element is accidentally square footage
                if 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:
                    
                    #make bedroom nan
                    bedroom_count = np.nan
                    num_bedroom.append(bedroom_count)
                    
                    #make sqft the first element
                    sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                    sqfts.append(sqft)
                    
                #if the length of the housing details element is more than 2
                elif len(post.find('span', class_ = 'housing').text.split()) > 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    num_bedroom.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                    sqfts.append(sqft)
                    
                #if there is num bedrooms but no sqft
                elif len(post.find('span', class_ = 'housing').text.split()) == 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    num_bedroom.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = np.nan
                    sqfts.append(sqft)                    
                
                else:
                    bedroom_count = np.nan
                    num_bedroom.append(bedroom_count)
                
                    sqft = np.nan
                    sqfts.append(sqft)
                
            #if none of those conditions catch, make bedroom nan, this won't be needed    
            else:
                bedroom_count = np.nan
                num_bedroom.append(bedroom_count)
                
                sqft = np.nan
                sqfts.append(sqft)
            #    num_bedroom.append(bedroom_count)
                
            #    sqft = np.nan
            #    sqfts.append(sqft)
        iterations += 1
        #print("Page" + str(iterations) + "Scraped Successfully!")   

Total number of search result: 3000
CPU times: total: 5.16 s
Wall time: 46.3 s


In [222]:
# create DataFrame
df = pd.DataFrame({'Post Datetime': post_date,
                   'Post Title': post_titles,
                   'Post URL': post_links,
                  'Neighborhood': post_cities,
                  'Bedroom': num_bedroom,
                  'SQFT': sqfts,
                  'Price': prices})

print(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Post Datetime  3000 non-null   object 
 1   Post Title     3000 non-null   object 
 2   Post URL       3000 non-null   object 
 3   Neighborhood   3000 non-null   object 
 4   Bedroom        2882 non-null   object 
 5   SQFT           2661 non-null   float64
 6   Price          3000 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 164.2+ KB
None


,Post Datetime,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price
0,2022-12-08 10:15,Well maintained whole house 3bed+3bath in West...,https://vancouver.craigslist.org/van/apa/d/van...,"Dunbar ,Vancouver",3,2107.0,4800
1,2022-12-08 10:14,1 furnished bedroom suit close Costco for rent!,https://vancouver.craigslist.org/rch/apa/d/ric...,richmond,1,NaN,1550
2,2022-12-08 10:14,1 Br APARTMENT - GREAT CENTRAL LOCATION,https://vancouver.craigslist.org/van/apa/d/van...,East Vancouver,1,NaN,1800
3,2022-12-08 10:13,1 Bedroom & Den Half Basement Suite (49th and ...,https://vancouver.craigslist.org/van/apa/d/van...,Killarney,1,748.0,1550
4,2022-12-08 10:10,Spacious townhouse available!,https://vancouver.craigslist.org/van/apa/d/van...,Yaletown,2,1510.0,3900


In [223]:
df.tail()

,Post Datetime,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price
2995,2022-12-05 17:11,1 bedroom + DEN!!! BRAND NEW FURNISHED MODERN...,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,NaN,2800
2996,2022-12-05 17:09,2 Bedroom 1 Bathroom at the Tate,https://vancouver.craigslist.org/van/apa/d/van...,"1283 Howe Street, Vancouver",2,615.0,3000
2997,2022-12-05 17:09,BRAND NEW Luxury 2 Br + Flex Condo (Willoughby,https://vancouver.craigslist.org/rds/apa/d/lan...,Langley,2,973.0,2500
2998,2022-12-05 17:07,Downtown Yaletown Westend $2195 - Large 1BR Ja...,https://vancouver.craigslist.org/van/apa/d/van...,950 Drake Street,1,650.0,2195
2999,2022-12-05 17:06,Luxurious Almost-brand-new 2 bedroom basement ...,https://vancouver.craigslist.org/van/apa/d/van...,Arbutus,2,NaN,2495


## Process

Check the dataset I created from the Craigslist ad and final adjustment of the dataset to analyze the data. 

First to check the dataset:

In [224]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Post Datetime  3000 non-null   object 
 1   Post Title     3000 non-null   object 
 2   Post URL       3000 non-null   object 
 3   Neighborhood   3000 non-null   object 
 4   Bedroom        2882 non-null   object 
 5   SQFT           2661 non-null   float64
 6   Price          3000 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 164.2+ KB


Convert Post Datetime object to Datetime

I would like to distingish FULLY FURNISHED suite and not furnished suite. I will create another column to show if it isTURE or FALSE. I noticed the title says if it is furnished suite or not (If not on the title, I assume that the suite is not furnished) 

Also, I noticed that the number of bedroom is string, so I will convert to integer. 

In [225]:
# Convert Datetime then split seperate columns
df['Post Datetime'] = pd.to_datetime(df["Post Datetime"])
df['Post Date'] = df['Post Datetime'].dt.date
df['Post Time'] = df['Post Datetime'].dt.time
df = df[['Post Datetime', 'Post Date', 'Post Time', 'Post Title', 'Post URL', 'Neighborhood', 'Bedroom', 'SQFT', 'Price']]
df.tail()

,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price
2995,2022-12-05 17:11:00,2022-12-05,17:11:00,1 bedroom + DEN!!! BRAND NEW FURNISHED MODERN...,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,NaN,2800
2996,2022-12-05 17:09:00,2022-12-05,17:09:00,2 Bedroom 1 Bathroom at the Tate,https://vancouver.craigslist.org/van/apa/d/van...,"1283 Howe Street, Vancouver",2,615.0,3000
2997,2022-12-05 17:09:00,2022-12-05,17:09:00,BRAND NEW Luxury 2 Br + Flex Condo (Willoughby,https://vancouver.craigslist.org/rds/apa/d/lan...,Langley,2,973.0,2500
2998,2022-12-05 17:07:00,2022-12-05,17:07:00,Downtown Yaletown Westend $2195 - Large 1BR Ja...,https://vancouver.craigslist.org/van/apa/d/van...,950 Drake Street,1,650.0,2195
2999,2022-12-05 17:06:00,2022-12-05,17:06:00,Luxurious Almost-brand-new 2 bedroom basement ...,https://vancouver.craigslist.org/van/apa/d/van...,Arbutus,2,NaN,2495


In [226]:
# Convert Bedroom number strings to Integer then NaN value to 0 assume 0 bedroom is studio appartment. 
df["Bedroom"] = pd.to_numeric(df["Bedroom"], errors='ignore').astype('Int64')
df["Bedroom"] = df["Bedroom"].fillna(0)

In [227]:
%%time
# Check if word 'furnished' in the title post. 
search_word = 'furnished'
furnished = []
for i in range(len(df)):
    post_title_list = df['Post Title'][i].lower().split(" ")
    if any(word == search_word for word in post_title_list):
        furnished.append(True)
    else:
        furnished.append(False)
df["Furnished?"] = furnished

CPU times: total: 15.6 ms
Wall time: 15 ms


Check the number of $0 posts then do something with them based on the result. 

In [228]:
# How many ads are posted less than $50?
posted_low_price = df[df['Price'] < 50]
print(f'Number of $0 post: {len(posted_low_price)}')
posted_low_price

Number of $0 post: 13


,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,Furnished?
779,2022-12-07 15:57:00,2022-12-07,15:57:00,SHORT TERM RENTAL. Daily/Weekly. Great locatio...,https://vancouver.craigslist.org/pml/apa/d/por...,Coquitlam Westwood Platea/ Eagle Mountain,2,800.0,9,False
1501,2022-12-07 08:27:00,2022-12-07,08:27:00,upper level Furnished suite in South Surrey,https://vancouver.craigslist.org/rds/apa/d/sur...,South Surrey/Ocean Park area,3,1350.0,1,True
1502,2022-12-07 08:27:00,2022-12-07,08:27:00,Short term rental upper level furnished suite ...,https://vancouver.craigslist.org/rch/apa/d/sur...,South Surrey/Ocean Park Area,3,1350.0,1,True
1508,2022-12-07 08:16:00,2022-12-07,08:16:00,Westsea Place - (Westend) - Studio & 1 Bedroom...,https://vancouver.craigslist.org/van/apa/d/del...,1644 NELSON ST.,0,NaN,0,False
1528,2022-12-07 07:19:00,2022-12-07,07:19:00,2 BED ROOM FOR RENT,https://vancouver.craigslist.org/bnc/apa/d/new...,NEW WESTMINSTER,2,NaN,0,False
1666,2022-12-06 22:50:00,2022-12-06,22:50:00,COUNTRY WOOD ESTATE HOME AVAILABLE FOR RENT,https://vancouver.craigslist.org/rds/apa/d/sur...,delta/surrey/langley,6,NaN,0,False
1671,2022-12-06 22:50:00,2022-12-06,22:50:00,COUNTRY WOOD ESTATE HOME AVAILABLE FOR RENT,https://vancouver.craigslist.org/rds/apa/d/sur...,delta/surrey/langley,6,NaN,0,False
1683,2022-12-06 22:38:00,2022-12-06,22:38:00,"SCAM -$1,500 / 1br - 755ft2 - 1BR in FREESIA B...",https://vancouver.craigslist.org/van/apa/d/van...,"1082 Seymour St, Vancouver, BC",0,NaN,0,False
1684,2022-12-06 22:37:00,2022-12-06,22:37:00,"SCAM/FAKE $1,500 1br - 755ft2 - in FREESIA Bui...",https://vancouver.craigslist.org/van/apa/d/van...,"1082 Seymour St, Vancouver, BC",0,NaN,0,False
1774,2022-12-06 20:58:00,2022-12-06,20:58:00,ASAP Laundry Service,https://vancouver.craigslist.org/van/apa/d/van...,Downtown Vancouver,0,NaN,0,False


In this case, only 13 total posts less than $50, so I will drop those rows. 

In [229]:
df.drop(index=posted_low_price.index, axis=0, inplace=True)
len(df)

2987

In [232]:
df.head(5)

,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,Furnished?
0,2022-12-08 10:15:00,2022-12-08,10:15:00,Well maintained whole house 3bed+3bath in West...,https://vancouver.craigslist.org/van/apa/d/van...,"Dunbar ,Vancouver",3,2107.0,4800,False
1,2022-12-08 10:14:00,2022-12-08,10:14:00,1 furnished bedroom suit close Costco for rent!,https://vancouver.craigslist.org/rch/apa/d/ric...,richmond,1,NaN,1550,True
2,2022-12-08 10:14:00,2022-12-08,10:14:00,1 Br APARTMENT - GREAT CENTRAL LOCATION,https://vancouver.craigslist.org/van/apa/d/van...,East Vancouver,1,NaN,1800,False
3,2022-12-08 10:13:00,2022-12-08,10:13:00,1 Bedroom & Den Half Basement Suite (49th and ...,https://vancouver.craigslist.org/van/apa/d/van...,Killarney,1,748.0,1550,False
4,2022-12-08 10:10:00,2022-12-08,10:10:00,Spacious townhouse available!,https://vancouver.craigslist.org/van/apa/d/van...,Yaletown,2,1510.0,3900,False


In [233]:
# save to csv file
df.to_csv('vancouver_rental_ads.csv', mode='a', header=False)

## Analyze

In [138]:
# Count how many rentals are furnished 
num_furnished_suite = df[df['Furnished?'] == True]["Furnished?"].count()
num_unfurnished_suite = len(df) - num_furnished_suite
furnished_suite_percent = np.round((num_furnished_suite / len(df)) * 100, 2)
print(f"Number of furnished suite is {num_furnished_suite}, and it's {furnished_suite_percent}% of the entire list.")

Number of furnished suite is 445, and it's 14.9% of the entire list.


In [139]:
# Find out the number of ads on each bedrooms number.  
num_bed = np.sort(df["Bedroom"].unique())
suite_counts = []
for i in range(len(num_bed)):
    suite_count = len(df[df['Bedroom'] == num_bed[i]])
    suite_counts.append(suite_count)
    #print(f'Number of {num_bed[i]} Bedroom suite: {len(df[df["Bedroom"] == num_bed[i]])}')
#print(f"List of suite counts: {suite_counts}")

In [140]:
# Find out Mean, Max, Min of SQFT and Price of each bedroom and SUM of furnished suite and total suite on each bedroom. 
df_count = df.groupby(by=['Bedroom']).agg({'SQFT': ['mean', 'max', 'min'],
                                                 'Price': ['mean', 'max', 'min'],
                                                  'Furnished?': ['sum', 'count']})
df_count['percent furnished'] = df_count['Furnished?']['sum'] / df_count["Furnished?"]['count']
df_count

SQFT                          Price               Furnished?  \
                mean      max     min          mean     max   min        sum   
Bedroom                                                                        
0         442.660000   1200.0   228.0   2155.539130    4050   375         27   
1         627.305170   2000.0   150.0   2341.563596    6500    90        165   
2        1062.255526  85000.0   300.0   3851.440352  500000   140        185   
3        1568.011204   4256.0   650.0   4586.460048   24995   138         42   
4        2553.928571   4900.0   900.0   5126.569832   23000   800         13   
5        3686.158537   7600.0  1796.0   7687.388235   39990  3100          9   
6        5909.950000  29090.0  2250.0   8208.250000   22880  3880          1   
7        4296.428571   6000.0  2900.0   8461.125000   19995  4800          1   
8        9745.833333  16294.0  4788.0  22915.833333   59995  5600          2   

              percent furnished  
        count                    
Bedroom                          
0         115          0.234783  
1         912          0.180921  
2        1249          0.148118  
3         413          0.101695  
4         179          0.072626  
5          85          0.105882  
6          20          0.050000  
7           8          0.125000  
8           6          0.333333

In [141]:
# Price per furnished suit on each bedroom
#df_price_furnished = df.drop(['Post Datetime', 'Post Title', 'Post URL', 'Neighborhood'], axis=1)
df_price_furnished = df.groupby(['Bedroom', 'Furnished?']).agg({'Price': ['mean', 'max', 'min', 'count'],
                                                               "SQFT": ['mean', 'max', 'min']})
df_price_furnished

Price                               SQFT           \
                            mean     max    min count          mean      max   
Bedroom Furnished?                                                             
0       False        2127.863636    4050    375    88    457.144737   1200.0   
        True         2245.740741    3000   1250    27    396.791667    550.0   
1       False        2233.338688    6500     90   747    627.744113   2000.0   
        True         2831.527273    6495    960   165    625.512821   1500.0   
2       False        3257.556391   28520    700  1064   1057.086458  85000.0   
        True         7267.075676  500000    140   185   1091.274854   2873.0   
3       False        4283.808625   19975    138   371   1546.578616   4256.0   
        True         7259.880952   24995   2400    42   1742.769231   3800.0   
4       False        4957.584337   23000    800   166   2522.865385   4900.0   
        True         7284.384615   12000   4200    13   2957.750000   4500.0   
5       False        7002.078947   20000   3100    76   3654.835616   7600.0   
        True        13474.444444   39990   6495     9   3940.222222   5271.0   
6       False        7436.052632   17000   3880    19   5694.684211  29090.0   
        True        22880.000000   22880  22880     1  10000.000000  10000.0   
7       False        6813.428571    9995   4800     7   4012.500000   5300.0   
        True        19995.000000   19995  19995     1   6000.000000   6000.0   
8       False        9375.000000   12000   5600     4   6484.000000   8084.0   
        True        49997.500000   59995  40000     2  16269.500000  16294.0   

                             
                        min  
Bedroom Furnished?           
0       False         228.0  
        True          228.0  
1       False         150.0  
        True          275.0  
2       False         300.0  
        True          550.0  
3       False         650.0  
        True          900.0  
4       False         900.0  
        True         1750.0  
5       False        1796.0  
        True         2500.0  
6       False        2250.0  
        True        10000.0  
7       False        2900.0  
        True         6000.0  
8       False        4788.0  
        True        16245.0

Dropped extrimely low price add, and it seems no outlier on the price

In [143]:
# Count how many ads per day. 
df.groupby('Post Date').agg('count')

,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,Furnished?
Post Date,,,,,,,,
2022-12-05,537,537,537,537,537,495,537,537
2022-12-06,1099,1099,1099,1099,1099,962,1099,1099
2022-12-07,1257,1257,1257,1257,1257,1119,1257,1257
2022-12-08,94,94,94,94,94,88,94,94


It looks like around 1000 posts per day on weekday. 

### Create dataFrame for just one day

In [165]:
# December 7, 2022's ads
df_one_day = df[df['Post Date'] == datetime(2022, 12, 7).date()]
df_one_day

,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,Furnished?
94,2022-12-07,23:58:00,"Large Luxury One Bedroom, S/Steel appliance, I...",https://vancouver.craigslist.org/bnc/apa/d/new...,New Westminster,1,622.0,2305,False
95,2022-12-07,23:57:00,BRAND NEW 3 Bedroom Suite for Rent in Panorama...,https://vancouver.craigslist.org/rds/apa/d/sur...,Surrey,3,2000.0,3400,False
96,2022-12-07,23:55:00,3 bed 3 bath plus 2 parking townhouse,https://vancouver.craigslist.org/van/apa/d/van...,King Edward and Cambie,3,1400.0,4500,False
97,2022-12-07,23:54:00,3 bedroom 3 bathrooms for rent,https://vancouver.craigslist.org/bnc/apa/d/ric...,Burnaby,3,1000.0,2790,False
98,2022-12-07,23:54:00,UPPER LEVEL 2 BEDROOM,https://vancouver.craigslist.org/van/apa/d/van...,city of vancouver,2,NaN,2500,False
...,...,...,...,...,...,...,...,...,...
1351,2022-12-07,00:18:00,Panoramic Oceanview 4br Home West Van-Whole House,https://vancouver.craigslist.org/van/apa/d/wes...,West Vancouver,4,2000.0,6000,False
1352,2022-12-07,00:17:00,BRAND NEW OCEAN VIEW SPACIOUS 2BR UNIT HIGH FLOOR,https://vancouver.craigslist.org/nvn/apa/d/coq...,North Vancouver,2,950.0,3600,False
1353,2022-12-07,00:04:00,2 bedroom house langley,https://vancouver.craigslist.org/rds/apa/d/lan...,Langley,2,NaN,2300,False
1354,2022-12-07,00:03:00,North facing 2 bedroom apartment with full siz...,https://vancouver.craigslist.org/bnc/apa/d/new...,"527 Carnarvon Street, New Westminster, BC",2,716.0,2745,False


## Share

## Act